# Robot Painter

(circa 11/23/22) Code taken from Elaine Pham's work for and the starter code from https://deepnote.com/workspace/Manipulation-ac8201a1-470a-4c77-afd0-2cc45bc229ff/project/18ba0481-7e30-46ba-9e7d-f7b34500d6dc/%2Frobot_painter.ipynb

intermediate function source (circa 11/25/22 https://stackoverflow.com/questions/43594646/how-to-calculate-the-coordinates-of-the-line-between-two-points-in-python)

Changes to make straight lines going to a center point instead of a circle and to save those.

In [1]:
import matplotlib.pyplot as plt
import mpld3
import numpy as np
import pandas as pd
from IPython.display import HTML, display
from manipulation import running_as_notebook, FindResource
from manipulation.meshcat_utils import AddMeshcatTriad
from manipulation.scenarios import MakeManipulationStation
from pydrake.all import (AddMultibodyPlantSceneGraph, AngleAxis, BasicVector,
                         ConstantVectorSource, DiagramBuilder,
                         FindResourceOrThrow, Integrator, JacobianWrtVariable,
                         LeafSystem, MeshcatVisualizer,
                         MeshcatVisualizerParams, MultibodyPlant,
                         MultibodyPositionToGeometryPose, Parser,
                         PiecewisePose, Quaternion, RigidTransform,
                         RollPitchYaw, RotationMatrix, SceneGraph, Simulator,
                         StartMeshcat, TrajectorySource)
import numpy as np

# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at https://a569df8b-a69d-4307-8cbd-4f24b608e3b4.deepnoteproject.com/7000/
Installing NginX server for MeshCat on Deepnote...


In [2]:
### Classes and helper functions from robot painter pset ###

class IIWA_Painter():
    def __init__(self, traj=None, init_angles=None):
        builder = DiagramBuilder()
        # set up the system of manipulation station
        self.station = MakeManipulationStation(filename=FindResource("models/iiwa_and_wsg.dmd.yaml"))

        builder.AddSystem(self.station)
        self.plant = self.station.GetSubsystemByName("plant")

        # optionally add trajectory source
        if traj is not None:
            traj_V_G = traj.MakeDerivative()
            V_G_source = builder.AddSystem(TrajectorySource(traj_V_G))
            self.controller = builder.AddSystem(
                PseudoInverseController(self.plant))
            builder.Connect(V_G_source.get_output_port(),
                            self.controller.GetInputPort("V_G"))

            self.integrator = builder.AddSystem(Integrator(7))
            builder.Connect(self.controller.get_output_port(),
                            self.integrator.get_input_port())
            builder.Connect(self.integrator.get_output_port(),
                            self.station.GetInputPort("iiwa_position"))
            builder.Connect(
                self.station.GetOutputPort("iiwa_position_measured"),
                self.controller.GetInputPort("iiwa_position"))

        params = MeshcatVisualizerParams()
        params.delete_on_initialization_event = False
        self.visualizer = MeshcatVisualizer.AddToBuilder(
            builder, self.station.GetOutputPort("query_object"), meshcat, params)

        wsg_position = builder.AddSystem(ConstantVectorSource([0.1]))
        builder.Connect(wsg_position.get_output_port(),
                        self.station.GetInputPort("wsg_position"))

        self.diagram = builder.Build()
        self.gripper_frame = self.plant.GetFrameByName('body')
        self.world_frame = self.plant.world_frame()

        context = self.CreateDefaultContext(init_angles)
        self.diagram.Publish(context)

    def visualize_frame(self, name, X_WF, length=0.15, radius=0.006):
        """
        visualize imaginary frame that are not attached to existing bodies
        
        Input: 
            name: the name of the frame (str)
            X_WF: a RigidTransform to from frame F to world.
        
        Frames whose names already exist will be overwritten by the new frame
        """
        AddMeshcatTriad(meshcat, "painter/" + name,
                        length=length, radius=radius, X_PT=X_WF)

    def CreateDefaultContext(self, init_angles=None):
        context = self.diagram.CreateDefaultContext()
        plant_context = self.diagram.GetMutableSubsystemContext(
            self.plant, context)
        station_context = self.diagram.GetMutableSubsystemContext(
            self.station, context)

        if init_angles is not None:
            q0 = init_angles
        else:
            # provide initial states
            q0 = np.array([ 1.40666193e-05,  1.56461165e-01, -3.82761069e-05,
                        -1.32296976e+00, -6.29097287e-06,  1.61181157e+00, -2.66900985e-05])
        # set the joint positions of the kuka arm
        iiwa = self.plant.GetModelInstanceByName("iiwa")
        self.plant.SetPositions(plant_context, iiwa, q0)
        self.plant.SetVelocities(plant_context, iiwa, np.zeros(7))
        wsg = self.plant.GetModelInstanceByName("wsg")
        self.plant.SetPositions(plant_context, wsg, [-0.05, 0.05])
        self.plant.SetVelocities(plant_context, wsg, [0, 0])        

        if hasattr(self, 'integrator'):
            self.integrator.set_integral_value(
                self.integrator.GetMyMutableContextFromRoot(context), q0)

        return context


    def get_X_WG(self, context=None):

        if not context:
            context = self.CreateDefaultContext()
        plant_context = self.plant.GetMyMutableContextFromRoot(context)
        X_WG = self.plant.CalcRelativeTransform(
                    plant_context,
                    frame_A=self.world_frame,
                    frame_B=self.gripper_frame)
        return X_WG

    def paint(self, sim_duration=20.0):
        context = self.CreateDefaultContext()
        simulator = Simulator(self.diagram, context)
        simulator.set_target_realtime_rate(1.0)

        duration = sim_duration if running_as_notebook else 0.01
        simulator.AdvanceTo(duration)

class PseudoInverseController(LeafSystem):
    """
    same controller seen in-class
    """
    def __init__(self, plant):
        LeafSystem.__init__(self)
        self._plant = plant
        self._plant_context = plant.CreateDefaultContext()
        self._iiwa = plant.GetModelInstanceByName("iiwa")
        self._G = plant.GetBodyByName("body").body_frame()
        self._W = plant.world_frame()

        self.V_G_port = self.DeclareVectorInputPort("V_G", BasicVector(6))
        self.q_port = self.DeclareVectorInputPort("iiwa_position", BasicVector(7))
        self.DeclareVectorOutputPort("iiwa_velocity", BasicVector(7),
                                     self.CalcOutput)
        self.iiwa_start = plant.GetJointByName("iiwa_joint_1").velocity_start()
        self.iiwa_end = plant.GetJointByName("iiwa_joint_7").velocity_start()

    def CalcOutput(self, context, output):
        V_G = self.V_G_port.Eval(context)
        q = self.q_port.Eval(context)
        self._plant.SetPositions(self._plant_context, self._iiwa, q)
        J_G = self._plant.CalcJacobianSpatialVelocity(
            self._plant_context, JacobianWrtVariable.kV,
            self._G, [0,0,0], self._W, self._W)
        J_G = J_G[:,self.iiwa_start:self.iiwa_end+1] # Only iiwa terms.
        v = np.linalg.pinv(J_G).dot(V_G) #important
        output.SetFromVector(v)

def compose_circular_key_frames(thetas, X_WorldCenter, X_WorldGripper_init, radius):
    """    
    returns: a list of RigidTransforms
    """

    ## this is an template, replace your code below
    # key_frame_poses_in_world = [X_WorldCenter @ X_WorldGripper_init]
    for i in range(len(thetas)):
        rotate = RigidTransform(RotationMatrix.MakeYRotation(thetas[i]));
        translate = RigidTransform([0, 0, -radius])
        this_pose = RigidTransform(X_WorldCenter @ rotate @ translate)
        key_frame_poses_in_world.append(this_pose)
        
    return key_frame_poses_in_world

# check key frames instead of interpolated trajectory
def visualize_key_frames(painter,frame_poses):
    for i, pose in enumerate(frame_poses):
        painter.visualize_frame('frame_{}'.format(i), pose, length=0.05)


In [3]:
### Elaine's helper function to generate straight line training data ###

def intermediates(p1, p2, nb_points):
    """"Return a list of nb_points equally spaced points
    between p1 and p2"""
    # If we have 8 intermediate points, we have 8+1=9 spaces
    # between p1 and p2
    x_spacing = (p2[0] - p1[0]) / (nb_points + 1)
    y_spacing = (p2[1] - p1[1]) / (nb_points + 1)
    z_spacing = (p2[2] - p1[2]) / (nb_points + 1)

    return [[p1[0] + i * x_spacing, p1[1] +  i * y_spacing, p1[2] +  i * z_spacing] 
            for i in range(1, nb_points+1)]

def compose_key_frames(X_WG_init, end_pnt, num_key_frames):
    """ create key frame poses with num_key_frames between the init gripper
    position and end gripper pos """
    X_WG_end = RigidTransform(X_WG_init.rotation(), end_pnt)
    results = [X_WG_init]
    key_pnts = intermediates(
                    X_WG_init.translation(), 
                    X_WG_end.translation(),
                    num_key_frames-1)
    for key_pnt in key_pnts:
        results.append(RigidTransform(X_WG_init.rotation(), np.array(key_pnt)))
    results.append(X_WG_end)
    
    return results

def run_simulation(times, key_frame_poses, trajectory_index, pose_index):
    """ create a traj from the key_frame_poses and run simulation """

    results = []

    # creates a trajectory out of the poses 
    traj = PiecewisePose.MakeLinear(times, key_frame_poses)
    # visualize the traj
    painter = IIWA_Painter(traj)
    # visualize_key_frames(painter, key_frame_poses)

    # run sim
    context = painter.CreateDefaultContext()
    plant_context = painter.plant.GetMyContextFromRoot(context)
    iiwa = painter.plant.GetModelInstanceByName("iiwa")
    simulator = Simulator(painter.diagram, context)
    simulator.set_target_realtime_rate(1.0)
    for timestep in times:
        simulator.AdvanceTo(timestep)
        gripper = painter.plant.GetBodyByName("iiwa_link_7")
        pose = np.ndarray.tolist(gripper.EvalPoseInWorld(plant_context).translation())
        angles = np.ndarray.tolist(painter.plant.GetPositions(plant_context, iiwa))
        current_pose_data = {
            'pose_index' : pose_index,
            'trajectory_index' : trajectory_index,
            'timestep' : timestep,
            'joint_angle' : angles,
            'end_effector_position' : pose,
        }
        results.append(current_pose_data)
        pose_index += 1
    
    # garbage collect 
    del painter; del context; del plant_context; del iiwa; del simulator
    return results, pose_index

def generate_line_data(end_pnts, gripper_init_pose, times, num_key_frames, start_traj_index=0):
    pose_index = 0 
    all_poses = []
    for trajectory_index in range(start_traj_index, len(end_pnts)):
        print('trajectory_index', trajectory_index)
        key_frames_pose = compose_key_frames(
                    gripper_init_pose,
                    end_pnts[trajectory_index],
                    num_key_frames=num_key_frames)
        results, pose_index = run_simulation(
                    times,
                    key_frames_pose,
                    trajectory_index, 
                    pose_index
        )
        del key_frames_pose
        all_poses.extend(results)

    pose_df = pd.DataFrame.from_records(all_poses)
    pd.to_pickle(pose_df, './diff_init_poses.pkl')

In [4]:
### Run this cell to generate straight line training data #######################

# simulation params
num_key_frames = 10
total_time = 20
times = np.linspace(0, total_time, num_key_frames+1)

# create iiwa and get init gripper pose
painter = IIWA_Painter()
gripper_init_pose = painter.get_X_WG()

# list of end gripper positions
end_pnts = []
for x in np.arange(.3, .5, .15):
    for y in np.arange(-.4, .3, .15):
        for z in np.arange(.28, .6, .15):
            end_pnts.append(np.array([x, y, z]))
print('num of trajectories', len(end_pnts))

generate_line_data(end_pnts, gripper_init_pose, times, num_key_frames)

num of trajectories 30
trajectory_index 0
trajectory_index 1
trajectory_index 2
trajectory_index 3
trajectory_index 4
trajectory_index 5
trajectory_index 6
trajectory_index 7
trajectory_index 8
trajectory_index 9
trajectory_index 10
trajectory_index 11
trajectory_index 12
trajectory_index 13
trajectory_index 14
trajectory_index 15
trajectory_index 16
trajectory_index 17
trajectory_index 18
trajectory_index 19
trajectory_index 20
trajectory_index 21
trajectory_index 22
trajectory_index 23
trajectory_index 24
trajectory_index 25
trajectory_index 26
trajectory_index 27
trajectory_index 28
trajectory_index 29


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=a569df8b-a69d-4307-8cbd-4f24b608e3b4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>